## Topographic Complexity/Variability: Fractal Dimension   
Developed in November 2023 by Dr. Larry Syu-Heng Lai (University of Washington)  

Recommended reference:
* Mark, D.M., Aronson, P.B., 1984. Scale-dependent fractal dimensions of topographic surfaces: An empirical investigation, with applications in geomorphology and computer mapping. Journal of the International Association for Mathematical Geology 16, 671-683. https://doi.org/10.1007/BF01033029
* Taud, H., Parrot, J.-F., 2006. Measurement of DEM roughness using the local fractal dimension. Géomorphologie : relief, processus, environnement 4. https://doi.org/10.4000/geomorphologie.622.
* Wilson, M.F.J., O’Connell, B., Brown, C., Guinan, J.C., Grehan, A.J., 2007. Multiscale Terrain Analysis of Multibeam Bathymetry Data for Habitat Mapping on the Continental Slope. Marine Geodesy 30, 3-35. https://doi.org/10.1080/01490410701295962 

### Initial setup

In [ ]:
import numpy as np
from numba import njit
from joblib import Parallel, delayed
import rasterio
from rasterio.windows import Window
from tqdm.notebook import tqdm

### Define data path

In [ ]:
# Define your file paths and file names separately
input_folder = '/Users/larryslai/Library/CloudStorage/Dropbox/QGIS/WA LiDAR/'
#input_file_name = 'Test_DEM.tif'
#input_file_name = 'Tokeland_DEM.tif'
#input_file_name = 'Nemah_DEM.tif'
input_file_name = 'Francies_DEM.tif'

output_folder = '/Users/larryslai/Library/CloudStorage/Dropbox/QGIS/WA LiDAR/'
#output_file_name = 'Test_pyFD.tif'
#output_file_name = 'Tokeland_pyFD.tif'
#output_file_name = 'Nemah_pyFD.tif'
output_file_name = 'Francies_pyFD.tif'

# Combine folder and file names to create the full paths
input_tif_path = input_folder + input_file_name
output_tif_path = output_folder + output_file_name

### Read a DEM

In [ ]:
# Open the input GeoTIFF file
with rasterio.open(input_tif_path) as src:
    # Read the first band (assumed to be elevation data)
    dem = src.read(1)
    # Retrieve the metadata from the source GeoTIFF to use for the output
    meta = src.meta

## Fractal Dimension (D)

Fractal dimension (D) is a measure of surface complexity, where higher values indicate more complex terrain. The calculation of fractal dimension using the variogram method consists of the following steps:

1. **Variogram Calculation**: The variogram is a function that quantifies the spatial variation of the terrain by calculating the semivariance of pixel elevation values at different lags (distances). The semivariance $ \gamma(h) $ for a lag $ h $ is computed using the formula:

$$
\gamma(h) = \frac{1}{2n(h)} \sum_{i=1}^{n} \sum_{j=1}^{n} (z_i - z_j)^2
$$

where:
- $ \gamma(h) $ is the semivariance at lag $ h $,
- $ n(h) $ is the number of pixel pairs at lag $ h $,
- $ z_i $, $ z_j $ are the elevation values of the pixel pairs.
- lag $h$ is the horizontal distance between the points/pixels

2. **Log-Log Regression**: The log-log regression is performed on the calculated variogram values. This involves plotting the log of the variogram values against the log of the lag distances and fitting a straight line to the points.

3. **Fractal Dimension Estimation**: The fractal dimension ($D$) thus provides a scalar value that characterizes the complexity of the terrain surface. A higher fractal dimension indicates a more complex and rough surface. The fractal dimension is estimated from the slope ($ m $) of the regression line obtained in the log-log plot. For the squared heigh differences ($(z_i - z_j)^2$) is computed for different distances, the relationship between slope $m$ and fractal dimension $D$ is (Mark & Aronson, 1984):

$$
D = 3 - \frac{m}{2}
$$

### Calculating local Fractal Dimension (D) to detect spatial variation in surface complexity

In heterogeneous landscapes, the fractal dimension can vary across the terrain. To capture this spatial variation, the fractal dimension is calculated locally for each pixel within a moving window. The local variogram method is applied, which computes the variogram for each cell based on its surrounding neighborhood defined by the window size. The log-log regression is then performed for each local window, resulting in a raster where each pixel's value represents the local fractal dimension of the surface around that pixel.

This approach allows for the assessment of spatial variations in terrain complexity, which can be crucial for applications such as habitat mapping and geomorphological analysis.

* Details methods follow Taud & Parrot (2006)

##### Functions

In [ ]:
def FDfunctions(window, hw):
    """
    Calculate the fractal dimension for the window using the box-counting method.
    
    :param window: 2D array of the moving window values.
    :param hw: Half of the window size.
    :return: Fractal dimension of the window.
    """
    # The center pixel's value
    center_pixel = window[hw, hw]

    # Calculate the number of voxels for each pixel in the window
    V = np.empty((window_size, window_size))
    count = 0
    for j in range(-hw, hw + 1):
        for k in range(-hw, hw + 1):
            T = window[hw + j, hw + k] - center_pixel
            if T < 0:
                V[hw + j, hw + k] = 0
            elif T > window_size:
                V[hw + j, hw + k] = window_size
            else:
                V[hw + j, hw + k] = T
            count += 1

    # Calculate the maximum number of voxels for varying box splitting
    list_box_sizes = [j for j in range(1, hw + 1) if hw % j == 0]
    Ns = np.empty(len(list_box_sizes))
    for i, q in enumerate(list_box_sizes):
        Ns[i] = max(V[k * q:(k + 1) * q, j * q:(j + 1) * q].sum() for k in range(window_size // q) for j in range(window_size // q)) / q
    
    # Avoid taking log of zero by adding a small epsilon where Ns is zero
    Ns = np.where(Ns == 0, np.nan, Ns)

    x = np.log(list_box_sizes)
    y = np.log(Ns)

    # Filter out non-finite values that may cause warnings
    finite_mask = np.isfinite(x) & np.isfinite(y)
    x = x[finite_mask]
    y = y[finite_mask]

    # Avoid regression if we have less than two points
    if 0 <= len(y) < 2:
        return np.nan
    
    m_x, m_y = np.mean(x), np.mean(y)
    SS_xy = np.sum(y * x) - len(x) * m_y * m_x
    SS_xx = np.sum(x * x) - len(x) * m_x * m_x

    slope = SS_xy / SS_xx

    # The fractal dimension D is the opposite of the slope
    D = 3 - (slope/2)
    return D

In [ ]:
def process_window(dem, i, j, hw):
    window = dem[i-hw:i+hw+1, j-hw:j+hw+1]
    return FDfunctions(window, hw)

##### Enable chunck processing to avoid RAM issues

In [ ]:
def dem_chunks_with_overlap(dem, window_size, chunk_size):
    hw = window_size // 2
    overlap = window_size // 2
    stride = chunk_size - overlap  # The effective stride

    # Calculate the number of chunks in each dimension
    num_chunks_y = (dem.shape[0] - overlap + stride - 1) // stride
    num_chunks_x = (dem.shape[1] - overlap + stride - 1) // stride

    for i_chunk in range(num_chunks_y):
        for j_chunk in range(num_chunks_x):
            # Calculate the start and end indices of the chunk
            i_start = i_chunk * stride
            j_start = j_chunk * stride
            i_end = i_start + chunk_size + overlap
            j_end = j_start + chunk_size + overlap

            # Ensure we don't go out of bounds on the last chunk
            i_end = min(i_end, dem.shape[0])
            j_end = min(j_end, dem.shape[1])

            # Extract the chunk
            chunk = dem[i_start:i_end, j_start:j_end]
            yield chunk, (i_start, j_start)


In [ ]:
def compute_fractal_dimensions_in_chunks(dem, window_size, chunk_size):
    fractal_dimension_map = np.full(dem.shape, np.nan, dtype=np.float32)
    hw = window_size // 2

    # Calculate total number of chunks for the progress bar
    total_chunks = (((dem.shape[0] - window_size) // (chunk_size - window_size + 1)) + 1) * \
                   (((dem.shape[1] - window_size) // (chunk_size - window_size + 1)) + 1)

    # Wrap the dem_chunks_with_overlap call with tqdm for a progress bar
    for chunk, (i_start, j_start) in tqdm(dem_chunks_with_overlap(dem, window_size, chunk_size), total=total_chunks, desc='Computing Fractal Dimensions'):
        local_indices = [(i, j) for i in range(hw, chunk.shape[0] - hw)
                               for j in range(hw, chunk.shape[1] - hw)]
        
        # Compute the fractal dimensions for the chunk
        fractal_dimensions = Parallel(n_jobs=-1)(
            delayed(process_window)(chunk, i, j, hw) for i, j in local_indices
        )

        # Write the computed fractal dimensions into the correct position in the map
        for ((i_local, j_local), fractal_dimension) in zip(local_indices, fractal_dimensions):
            i_global = i_start + i_local - hw
            j_global = j_start + j_local - hw
            if 0 <= i_global < fractal_dimension_map.shape[0] and 0 <= j_global < fractal_dimension_map.shape[1]:
                fractal_dimension_map[i_global, j_global] = fractal_dimension

    return fractal_dimension_map

In [ ]:
def read_input_geotiff(input_tiff_path):
    with rasterio.open(input_tiff_path) as src:
        dem = src.read(1)
        profile = src.profile
    return dem, profile

def write_output_geotiff(fractal_dimension_map, profile, output_tiff_path):
    profile.update(dtype=rasterio.float32, compress='lzw', tiled=True, bigtiff='IF_SAFER')
    with rasterio.open(output_tiff_path, 'w', **profile) as dst:
        dst.write(fractal_dimension_map.astype(rasterio.float32), 1)

##### Output data into GeoTIFF  

In [ ]:
window_size = 15  # The desired window size for the fractal dimension calculation (odd numbers)
chunk_size = 1024  # Define the chunk size without overlap

# Read the input GeoTIFF
dem, profile = read_input_geotiff(input_tif_path)

# Compute the fractal dimensions in chunks with overlap
fractal_dimension_map = compute_fractal_dimensions_in_chunks(dem, window_size, chunk_size)


In [ ]:
# Write the results to the output GeoTIFF
write_output_geotiff(fractal_dimension_map, profile, output_tif_path)